In [30]:
# This file will be used to visualize transformer attention dumps
import pickle, pdb
import sys
sys.path.append('..')
import utils
from collections import defaultdict as ddict
import numpy as np
from helper import *
from scipy.stats import ttest_ind

In [2]:
#import torch_geometric

In [3]:
#from torch_geometric.utils import add_remaining_self_loops

In [5]:
fst_res = load_pickle('../bot/predictions/basic_final_fst_seller_BERT_28_06_2020_06:01:00_all_metrics_test.pkl')
hed_res = load_pickle('../bot/predictions/basic_final_none_seller_BERT_28_06_2020_05:48:40_all_metrics_test.pkl')
transformer_res = load_pickle('../bot/predictions/basic_final_transformer_seller_BERT_28_06_2020_06:01:00_all_metrics_test.pkl')
graph_res = load_pickle('../bot/predictions/basic_final_graph_seller_gat_64_08_2_001_BERT_28_06_2020_06:01:24_all_metrics_test.pkl')
rnn_res = load_pickle('../bot/predictions/basic_final_rnn_seller_BERT_64_30_06_2020_01:00:15_all_metrics_test.pkl')

In [22]:
len(graph_res['all_metrics'])#.keys()
results = {'fst': fst_res, 'hed': hed_res, 'transformer': transformer_res, 'graph': graph_res, 'rnn': rnn_res}
for k,v in results.items():
    print (k, len(v['all_metrics']))
    print (v.keys())

fst 50
dict_keys(['loss', 'metrics', 'all_metrics'])
hed 50
dict_keys(['loss', 'metrics', 'all_metrics'])
transformer 50
dict_keys(['loss', 'metrics', 'all_metrics'])
graph 50
dict_keys(['loss', 'metrics', 'all_metrics'])
rnn 50
dict_keys(['loss', 'metrics', 'all_metrics'])


In [20]:
print (graph_res['all_metrics'][0]['scores'])

{'bleu': 0.23665738346359966, 'bert_scores_P': 0.26794856786727905, 'bert_scores_R': 0.28667518496513367, 'bert_scores_F1': 0.27758312225341797, 'ratio_acc': 0.5454545454545454, 'da_acc': 0.42452830188679247, 'strat_report': '                             precision    recall  f1-score   support\n\n                   agent_id       0.50      1.00      0.67        53\n            assertive_count       0.00      0.00      0.00         5\n      third_person_singular       0.35      0.57      0.44        30\n                hedge_count       0.19      0.20      0.19        15\n              neg_sentiment       0.20      0.67      0.31         9\n                   trade_in       0.05      1.00      0.09         2\n              liwc_informal       0.00      0.00      0.00         0\n           personal_concern       0.13      1.00      0.23        11\n                     friend       0.02      1.00      0.03         1\n             liwc_certainty       0.09      1.00      0.16         6\nfi

In [24]:
flatten_res = ddict(lambda : ddict(list))
for model, mod in results.items():
    for batch in mod['all_metrics']:
        for metric, val in batch['scores'].items():
            if 'report' not in metric:
                flatten_res[model][metric].append(val) # val is a point
for k in flatten_res.keys():
    flatten_res[k] = dict(flatten_res[k])
flatten_res = dict(flatten_res)
print (flatten_res.keys())
print (flatten_res['graph'].keys())
print (len(flatten_res['graph']['strat_f1_macro']))

dict_keys(['fst', 'hed', 'transformer', 'graph', 'rnn'])
dict_keys(['bleu', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1', 'ratio_acc', 'da_acc', 'strat_f1_macro', 'da_f1_macro', 'strat_f1_micro', 'da_f1_micro', 'strat_f1_weighted', 'da_f1_weighted'])
50


In [29]:
a = flatten_res['graph']['bert_scores_F1']
b = flatten_res['fst']['bert_scores_F1']
aa = []
bb = []
for x in results['graph']['all_metrics']:
    aa += x['bertf1']
for x in results['fst']['all_metrics']:
    bb += x['bertf1']
#aa = results['graph']['all_metrics']['bert_scores_F1']
#bb = results['fst']['all_metrics']['bert_scores_F1']
print (len(a), len(b), len(aa), len(bb))

50 50 5548 5548


In [31]:
print(ttest_ind(a, b))
print (ttest_ind(aa, bb))

Ttest_indResult(statistic=1.2069210324629778, pvalue=0.23036773695563917)
Ttest_indResult(statistic=1.1510675861806314, pvalue=0.24972923943532593)


In [35]:
def test(model1, model2, metric):
    a = flatten_res[model1][metric]
    b = flatten_res[model2][metric]
    print (len(a), len(b))
    return ttest_ind(a,b)
models = list(flatten_res.keys())
metrics = list(flatten_res['graph'].keys())
for model1 in models:
    for model2 in models:
        if model1 == model2: continue
        print ('\n\nModel 1 : {}, Model 2 : {}'.format(model1, model2))
        for metric in metrics:
#             if metric != 'bert_scores_F1': continue
            print ('Metric : ', metric)
            res = test(model1, model2, metric)
            print (res.pvalue)
#             aa = []
#             bb = []
#             for x in results[model1]['all_metrics']:
#                 aa += x['bertf1']
#             for x in results[model2]['all_metrics']:
#                 bb += x['bertf1']
#             print (ttest_ind(aa, bb).pvalue)



Model 1 : fst, Model 2 : hed
Metric :  bleu
50 50
1.7130497523690057e-06
Metric :  bert_scores_P
50 50
1.7064911957389247e-11
Metric :  bert_scores_R
50 50
1.5700429183747772e-10
Metric :  bert_scores_F1
50 50
1.757152854319192e-11
Metric :  ratio_acc
50 50
0.026696343780008756
Metric :  da_acc
50 50
1.3990791572703337e-79
Metric :  strat_f1_macro
50 50
5.160075429641834e-87
Metric :  da_f1_macro
50 50
4.244347905046861e-66
Metric :  strat_f1_micro
50 50
9.340458929532454e-94
Metric :  da_f1_micro
50 50
1.3990791572703337e-79
Metric :  strat_f1_weighted
50 50
2.907264758854915e-101
Metric :  da_f1_weighted
50 50
4.710983729501361e-70


Model 1 : fst, Model 2 : transformer
Metric :  bleu
50 50
0.22730255706620525
Metric :  bert_scores_P
50 50
0.6544280768797437
Metric :  bert_scores_R
50 50
0.113005720783591
Metric :  bert_scores_F1
50 50
0.3011287122681977
Metric :  ratio_acc
50 50
0.002578407528829729
Metric :  da_acc
50 50
2.0415446934250964e-13
Metric :  strat_f1_macro
50 50
4.880

# Bootstrap method

In [89]:
fst_res = load_pickle('../bot/predictions/basic_final_fst_seller_BERT_28_06_2020_06:01:00_all_metrics_test.pkl')
hed_res = load_pickle('../bot/predictions/basic_final_none_seller_BERT_28_06_2020_05:48:40_all_metrics_test.pkl')
transformer_res = load_pickle('../bot/predictions/basic_final_transformer_seller_BERT_28_06_2020_06:01:00_all_metrics_test.pkl')
graph_res = load_pickle('../bot/predictions/basic_final_graph_seller_gat_64_08_2_001_BERT_28_06_2020_06:01:24_all_metrics_test.pkl')
rnn_res = load_pickle('../bot/predictions/basic_final_rnn_seller_BERT_64_30_06_2020_01:00:15_all_metrics_test.pkl')

In [90]:
graph_res['all_metrics'].keys()

dict_keys(['all_ratio_preds', 'all_ratio_buckets', 'all_strat_preds', 'all_strat_tgts', 'all_da_preds', 'all_da_tgts', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1', 'bleus'])

In [91]:
new_results = {'fst': fst_res, 'hed': hed_res, 'transformer': transformer_res, 'graph': graph_res, 'rnn': rnn_res}

In [92]:
for metric in ['bleus', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1']:
    print()
    print (metric)
    i = 0
    while i < len(models)-1:
        model1 = models[i]
        j = i+1
        while j < len(models):
            model2 = models[j]
            res = ttest_ind(new_results[model1]['all_metrics'][metric], new_results[model2]['all_metrics'][metric])
            print (model1, model2, res.pvalue)
            j += 1
        i += 1


bleus
hed rnn 0.016736102175444943
hed fst 1.2530585201679614e-05
hed transformer 6.638873695009778e-08
hed graph 8.56308714427927e-09
rnn fst 0.055079555890115366
rnn transformer 0.003175751570314486
rnn graph 0.0009288493277561732
fst transformer 0.2918505984605315
fst graph 0.1538100535042362
transformer graph 0.7083469787144858

bert_scores_P
hed rnn 0.18886093738323023
hed fst 7.996749011928258e-14
hed transformer 3.84286392448271e-15
hed graph 5.805920376170552e-16
rnn fst 2.6002106218235895e-09
rnn transformer 2.265521555129666e-10
rnn graph 4.643125805340287e-11
fst transformer 0.6700028230358002
fst graph 0.4737521758574834
transformer graph 0.7702059652600827

bert_scores_R
hed rnn 0.7380791583541548
hed fst 4.9303032498839275e-11
hed transformer 6.414594504410869e-16
hed graph 5.094150360744307e-16
rnn fst 1.3253140134556394e-09
rnn transformer 5.112465190174902e-14
rnn graph 4.015898151013183e-14
fst transformer 0.14099658980185695
fst graph 0.12425482741894793
transformer

In [93]:
# Do "np.concatenate(preds, 0)" for strategy and da preds and targets
for model in models:
    if model == 'hed': continue
    new_results[model]['all_metrics']['all_strat_preds'] = np.concatenate(new_results[model]['all_metrics']['all_strat_preds'], 0)
    new_results[model]['all_metrics']['all_strat_tgts'] = np.concatenate(new_results[model]['all_metrics']['all_strat_tgts'], 0)
    new_results[model]['all_metrics']['all_da_preds'] = np.concatenate(new_results[model]['all_metrics']['all_da_preds'], 0)
    new_results[model]['all_metrics']['all_da_tgts'] = np.concatenate(new_results[model]['all_metrics']['all_da_tgts'], 0).flatten()
    

In [102]:
from scipy.special import softmax
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def get_score(preds, targets, metric):
    if metric == 'ratio_acc':
        preds = np.array(preds)
        targets = np.array(targets)
        score = np.sum(preds == targets) / len(preds)
        return score
    elif 'strat' in metric:
        preds = sigmoid(preds)
        pred_scores = copy.deepcopy(preds)
        preds[preds >= 0.5] = 1
        preds[preds < 0.5]  = 0
    elif 'da' in metric:
        preds = softmax(preds, axis = 1)
        pred_scores = copy.deepcopy(preds)
        preds = np.argmax(preds, axis = 1)
    averaging = metric.split('_')[-1]
    if 'f1' in metric:
        score = f1_score(targets, preds, average=averaging)
    else:
        if 'strat' in metric:
            score = roc_auc_score(targets, pred_scores, average = averaging)
        else:
            if averaging == 'micro':
                score = roc_auc_score(targets, pred_scores, average = averaging)
            else:
                score = roc_auc_score(targets, pred_scores, average = averaging, multi_class = 'ovr')
    return score

In [54]:
n = new_results[model]['all_metrics']['all_strat_preds'].shape[0]
b = 1000

In [55]:
print (n)

5548


In [99]:
models = ['hed', 'rnn', 'fst', 'transformer', 'graph']
for metric in ['bleus', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1', 'ratio_acc', 'strat_f1_macro', 'strat_f1_micro', 'strat_f1_weighted', 'strat_roc_auc_macro', 
              'strat_roc_auc_micro', 'strat_roc_auc_weighted', 'da_f1_macro', 'da_f1_micro', 'da_f1_weighted', 
              'da_roc_auc_macro', 'da_roc_auc_weighted']:
    print()
    if metric in ['bleus', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1', 'ratio_acc']: skip_hed = False
    else: skip_hed = True
    print (metric)
    i = 0
    while i < len(models):
        model1 = models[i]
        j = 0#i+1
        if skip_hed and model1 == 'hed':
            i += 1
            continue
        while j < len(models):
            model2 = models[j]
            if skip_hed and model2 == 'hed' or model1 == model2:
                j += 1
                continue
            r, resampled_metric_deltas, r_deltas = 0, [], []
            rr = 0
            for _ in range(b):
                indices = np.random.randint(0, n, n)
                if metric in ['bleus', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1']:
                    resampled_model1_score = np.mean(np.array(new_results[model1]['all_metrics'][metric])[indices])
                    resampled_model2_score = np.mean(np.array(new_results[model2]['all_metrics'][metric])[indices])
                    ref_delta = np.mean(new_results[model2]['all_metrics'][metric]) - np.mean(new_results[model1]['all_metrics'][metric])
                elif metric == 'ratio_acc':
                    indices = np.random.randint(0, 567, 567)
                    resampled_model1_preds = np.array(new_results[model1]['all_metrics']['all_ratio_preds'])[indices]
                    resampled_model1_tgts  = np.array(new_results[model1]['all_metrics']['all_ratio_buckets'])[indices]
                    resampled_model2_preds = np.array(new_results[model2]['all_metrics']['all_ratio_preds'])[indices]
                    resampled_model2_tgts  = np.array(new_results[model2]['all_metrics']['all_ratio_buckets'])[indices]
                    resampled_model1_score = get_score(resampled_model1_preds, resampled_model1_tgts, metric)
                    resampled_model2_score = get_score(resampled_model2_preds, resampled_model2_tgts, metric)
                    main_score1 = get_score(new_results[model1]['all_metrics']['all_ratio_preds'], new_results[model1]['all_metrics']['all_ratio_buckets'], metric)
                    main_score2 = get_score(new_results[model2]['all_metrics']['all_ratio_preds'], new_results[model2]['all_metrics']['all_ratio_buckets'], metric)
                    ref_delta = main_score2 - main_score1
                elif 'strat' in metric:
                    resampled_model1_preds = new_results[model1]['all_metrics']['all_strat_preds'][indices, :]
                    resampled_model1_tgts  = new_results[model1]['all_metrics']['all_strat_tgts'][indices, :]
                    resampled_model2_preds = new_results[model2]['all_metrics']['all_strat_preds'][indices, :]
                    resampled_model2_tgts  = new_results[model2]['all_metrics']['all_strat_tgts'][indices, :]
                    resampled_model1_score = get_score(resampled_model1_preds, resampled_model1_tgts, metric)
                    resampled_model2_score = get_score(resampled_model2_preds, resampled_model2_tgts, metric)
                    main_score1 = get_score(new_results[model1]['all_metrics']['all_strat_preds'], new_results[model1]['all_metrics']['all_strat_tgts'], metric)
                    main_score2 = get_score(new_results[model2]['all_metrics']['all_strat_preds'], new_results[model2]['all_metrics']['all_strat_tgts'], metric)
                    ref_delta = main_score2 - main_score1
                else:
                    resampled_model1_preds = new_results[model1]['all_metrics']['all_da_preds'][indices]
                    resampled_model1_tgts  = new_results[model1]['all_metrics']['all_da_tgts'][indices]
                    resampled_model2_preds = new_results[model2]['all_metrics']['all_da_preds'][indices]
                    resampled_model2_tgts  = new_results[model2]['all_metrics']['all_da_tgts'][indices]
                    resampled_model1_score = get_score(resampled_model1_preds, resampled_model1_tgts, metric)
                    resampled_model2_score = get_score(resampled_model2_preds, resampled_model2_tgts, metric)
                    main_score1 = get_score(new_results[model1]['all_metrics']['all_da_preds'], new_results[model1]['all_metrics']['all_da_tgts'], metric)
                    main_score2 = get_score(new_results[model2]['all_metrics']['all_da_preds'], new_results[model2]['all_metrics']['all_da_tgts'], metric)
                    ref_delta = main_score2 - main_score1
                resampled_metric_delta = resampled_model2_score - resampled_model1_score
                resampled_metric_deltas.append(resampled_metric_delta)
                if resampled_metric_delta >= 2 * ref_delta:
                    r += 1
                    r_deltas.append(resampled_metric_delta)
                if resampled_metric_delta > 0:
                    rr += 1
            p_val = float(r) / b
            print (model1, model2, p_val, 1 - float(rr) / b)
                
            j += 1
        i += 1


bleus
hed rnn 0.0 0.0
hed fst 0.0 0.0
hed transformer 0.0 0.0
hed graph 0.0 0.0
rnn hed 1.0 1.0
rnn fst 0.0 0.0
rnn transformer 0.0 0.0
rnn graph 0.0 0.0
fst hed 1.0 1.0
fst rnn 1.0 1.0
fst transformer 0.0 0.0
fst graph 0.0 0.0
transformer hed 1.0 1.0
transformer rnn 1.0 1.0
transformer fst 1.0 0.999
transformer graph 0.061 0.07499999999999996
graph hed 1.0 1.0
graph rnn 1.0 1.0
graph fst 1.0 1.0
graph transformer 0.931 0.933

bert_scores_P
hed rnn 0.0 0.0
hed fst 0.0 0.0
hed transformer 0.0 0.0
hed graph 0.0 0.0
rnn hed 1.0 1.0
rnn fst 0.0 0.0
rnn transformer 0.0 0.0
rnn graph 0.0 0.0
fst hed 1.0 1.0
fst rnn 1.0 1.0
fst transformer 0.109 0.10699999999999998
fst graph 0.025 0.019000000000000017
transformer hed 1.0 1.0
transformer rnn 1.0 1.0
transformer fst 0.894 0.88
transformer graph 0.193 0.17600000000000005
graph hed 1.0 1.0
graph rnn 1.0 1.0
graph fst 0.974 0.976
graph transformer 0.808 0.8240000000000001

bert_scores_R
hed rnn 0.187 0.18500000000000005
hed fst 0.0 0.0
hed transf

ValueError: multi_class must be in ('ovo', 'ovr')

In [104]:
models = ['hed', 'rnn', 'fst', 'transformer', 'graph']
for metric in ['strat_roc_auc_macro', 'strat_roc_auc_micro', 'strat_roc_auc_weighted',
              'da_roc_auc_macro', 'da_roc_auc_weighted']:
    print()
    if metric in ['bleus', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1', 'ratio_acc']: skip_hed = False
    else: skip_hed = True
    print (metric)
    i = 0
    while i < len(models):
        model1 = models[i]
        j = 0#i+1
        if skip_hed and model1 == 'hed':
            i += 1
            continue
        while j < len(models):
            model2 = models[j]
            if skip_hed and model2 == 'hed' or model1 == model2:
                j += 1
                continue
            r, resampled_metric_deltas, r_deltas = 0, [], []
            rr = 0
            for _ in range(b):
                indices = np.random.randint(0, n, n)
                if metric in ['bleus', 'bert_scores_P', 'bert_scores_R', 'bert_scores_F1']:
                    resampled_model1_score = np.mean(np.array(new_results[model1]['all_metrics'][metric])[indices])
                    resampled_model2_score = np.mean(np.array(new_results[model2]['all_metrics'][metric])[indices])
                    ref_delta = np.mean(new_results[model2]['all_metrics'][metric]) - np.mean(new_results[model1]['all_metrics'][metric])
                elif metric == 'ratio_acc':
                    indices = np.random.randint(0, 567, 567)
                    resampled_model1_preds = np.array(new_results[model1]['all_metrics']['all_ratio_preds'])[indices]
                    resampled_model1_tgts  = np.array(new_results[model1]['all_metrics']['all_ratio_buckets'])[indices]
                    resampled_model2_preds = np.array(new_results[model2]['all_metrics']['all_ratio_preds'])[indices]
                    resampled_model2_tgts  = np.array(new_results[model2]['all_metrics']['all_ratio_buckets'])[indices]
                    resampled_model1_score = get_score(resampled_model1_preds, resampled_model1_tgts, metric)
                    resampled_model2_score = get_score(resampled_model2_preds, resampled_model2_tgts, metric)
                    main_score1 = get_score(new_results[model1]['all_metrics']['all_ratio_preds'], new_results[model1]['all_metrics']['all_ratio_buckets'], metric)
                    main_score2 = get_score(new_results[model2]['all_metrics']['all_ratio_preds'], new_results[model2]['all_metrics']['all_ratio_buckets'], metric)
                    ref_delta = main_score2 - main_score1
                elif 'strat' in metric:
                    resampled_model1_preds = new_results[model1]['all_metrics']['all_strat_preds'][indices, :]
                    resampled_model1_tgts  = new_results[model1]['all_metrics']['all_strat_tgts'][indices, :]
                    resampled_model2_preds = new_results[model2]['all_metrics']['all_strat_preds'][indices, :]
                    resampled_model2_tgts  = new_results[model2]['all_metrics']['all_strat_tgts'][indices, :]
                    resampled_model1_score = get_score(resampled_model1_preds, resampled_model1_tgts, metric)
                    resampled_model2_score = get_score(resampled_model2_preds, resampled_model2_tgts, metric)
                    main_score1 = get_score(new_results[model1]['all_metrics']['all_strat_preds'], new_results[model1]['all_metrics']['all_strat_tgts'], metric)
                    main_score2 = get_score(new_results[model2]['all_metrics']['all_strat_preds'], new_results[model2]['all_metrics']['all_strat_tgts'], metric)
                    ref_delta = main_score2 - main_score1
                else:
                    resampled_model1_preds = new_results[model1]['all_metrics']['all_da_preds'][indices]
                    resampled_model1_tgts  = new_results[model1]['all_metrics']['all_da_tgts'][indices]
                    resampled_model2_preds = new_results[model2]['all_metrics']['all_da_preds'][indices]
                    resampled_model2_tgts  = new_results[model2]['all_metrics']['all_da_tgts'][indices]
                    resampled_model1_score = get_score(resampled_model1_preds, resampled_model1_tgts, metric)
                    resampled_model2_score = get_score(resampled_model2_preds, resampled_model2_tgts, metric)
                    main_score1 = get_score(new_results[model1]['all_metrics']['all_da_preds'], new_results[model1]['all_metrics']['all_da_tgts'], metric)
                    main_score2 = get_score(new_results[model2]['all_metrics']['all_da_preds'], new_results[model2]['all_metrics']['all_da_tgts'], metric)
                    ref_delta = main_score2 - main_score1
                resampled_metric_delta = resampled_model2_score - resampled_model1_score
                resampled_metric_deltas.append(resampled_metric_delta)
                if resampled_metric_delta >= 2 * ref_delta:
                    r += 1
                    r_deltas.append(resampled_metric_delta)
                if resampled_metric_delta > 0:
                    rr += 1
            p_val = float(r) / b
            print (model1, model2, p_val, 1 - float(rr) / b)
                
            j += 1
        i += 1


strat_roc_auc_macro
rnn fst 1.0 1.0
rnn transformer 0.0 0.0
rnn graph 0.0 0.0
fst rnn 0.0 0.0
fst transformer 0.0 0.0
fst graph 0.0 0.0
transformer rnn 1.0 1.0
transformer fst 1.0 1.0
transformer graph 0.657 0.665
graph rnn 1.0 1.0
graph fst 1.0 1.0
graph transformer 0.353 0.33099999999999996

strat_roc_auc_micro
rnn fst 1.0 1.0
rnn transformer 0.0 0.0
rnn graph 0.0 0.0
fst rnn 0.0 0.0
fst transformer 0.0 0.0
fst graph 0.0 0.0
transformer rnn 1.0 1.0
transformer fst 1.0 1.0
transformer graph 0.0 0.0
graph rnn 1.0 1.0
graph fst 1.0 1.0
graph transformer 1.0 1.0

strat_roc_auc_weighted
rnn fst 1.0 1.0
rnn transformer 0.0 0.0
rnn graph 0.0 0.0
fst rnn 0.0 0.0
fst transformer 0.0 0.0
fst graph 0.0 0.0
transformer rnn 1.0 1.0
transformer fst 1.0 1.0
transformer graph 0.626 0.642
graph rnn 1.0 1.0
graph fst 1.0 1.0
graph transformer 0.344 0.384

da_roc_auc_macro
rnn fst 1.0 1.0
rnn transformer 0.0 0.0
rnn graph 0.0 0.0
fst rnn 0.0 0.0
fst transformer 0.0 0.0
fst graph 0.0 0.0
transformer rn